# Item-based Models

In this file we are going to proceed with the building in the models. Machine learning models to build recommender systems. The models that are going to be built are Collaborative filtering using Item-based rating prediction (ItemKNN) and Item-based classification (ItemKNN).

Collaborative filtering is a technique used in recommendation systems to predict or classify items based on the preferences or behavior of similar users or items. Item-based Collaborative Filtering (CF) focuses on the similarity between items rather than users. There are two main approaches within item-based CF: rating prediction and classification.

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to make jupyter print all outputs, not just the last one
from IPython.core.display import HTML # to pretty print pandas df and be able to copy them over (e.g. to ppt slides)

In [3]:
# We import the necessary libraries
import pandas as pd
import numpy as np
import os

In [4]:
# We print the directory where the file is located
print(os.getcwd())

c:\Users\Jaume\Documents\MDDB\SDM\SDfM---Jaume-and-Stijn 2\SDfM---Jaume-and-Stijn


In [5]:
# We set the directory to the cleaned folder
os.listdir(os.path.join('.', 'cleaned'))

['final_sample30_parquet',
 'final_sample50_parquet',
 'final_sample5_parquet',
 'movielens_parquet',
 'netflix_parquet',
 'sample_tenth_netflix',
 'unpacked_reviews_df_100k.parquet']

In [6]:
# We read the final_sample file and store it in a dataframe
df = pd.read_parquet('cleaned/final_sample5_parquet')

In [7]:
# We print shape of the dataframe
df.shape

(4665, 26)

In [8]:
# We print the first 5 rows of the dataframe
df.head()

,movieId,title,year,userId,rating,date,Drama,Action,Sci-Fi,Comedy,...,Adventure,Fantasy,IMAX,Animation,Musical,Horror,Film-Noir,Western,Mystery,(no genres listed)
0,45635,"Notorious Bettie Page, The",2005,414,3.0,2008-07-15,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,45635,"Notorious Bettie Page, The",2005,474,3.0,2006-12-08,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1373,Star Trek V: The Final Frontier,1989,19,1.0,2000-08-08,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1373,Star Trek V: The Final Frontier,1989,42,4.0,2001-07-27,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1373,Star Trek V: The Final Frontier,1989,51,5.0,2009-01-02,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# We print the columns of the dataframe
df.columns

Index(['movieId', 'title', 'year', 'userId', 'rating', 'date', 'Drama',
       'Action', 'Sci-Fi', 'Comedy', 'Crime', 'Thriller', 'Romance', 'War',
       'Documentary', 'Children', 'Adventure', 'Fantasy', 'IMAX', 'Animation',
       'Musical', 'Horror', 'Film-Noir', 'Western', 'Mystery',
       '(no genres listed)'],
      dtype='object')

In [10]:
# We print the number of unique users and movies
print(df['userId'].nunique())
print(df['movieId'].nunique())


532
487


In [11]:
# Check if there are userIDs that haven't rated movies at all
print(df['userId'].value_counts().sort_values(ascending=False))

# Check if there are movieIDs that haven't been rated at all
print(df['movieId'].value_counts().sort_values(ascending=False))

userId
599    130
474    127
414    125
448     91
274     68
      ... 
101      1
13       1
157      1
145      1
598      1
Name: count, Length: 532, dtype: int64
movieId
1         215
4306      170
1580      165
104        99
551        94
         ... 
2037        1
3073        1
174551      1
54274       1
4966        1
Name: count, Length: 487, dtype: int64


In [12]:
# Get the rating for movieID 4966
df[df['movieId'] == 4966]

,movieId,title,year,userId,rating,date,Drama,Action,Sci-Fi,Comedy,...,Adventure,Fantasy,IMAX,Animation,Musical,Horror,Film-Noir,Western,Mystery,(no genres listed)
2914,4966,"Incredible Shrinking Man, The",1957,288,3.0,2002-07-10,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Check if there are NaN values in the rating column
df['rating'].isna().sum()

13

In [14]:
# Print the number of unique users
print("Number of unique users:", df['userId'].nunique())

# Print the number of unique movies
print("Number of unique movies:", df['movieId'].nunique())


Number of unique users: 532
Number of unique movies: 487


In [15]:
# print the data types of the DF
df.dtypes

movieId                 int64
title                  object
year                   object
userId                  int64
rating                float64
date                   object
Drama                   int64
Action                  int64
Sci-Fi                  int64
Comedy                  int64
Crime                   int64
Thriller                int64
Romance                 int64
War                     int64
Documentary             int64
Children                int64
Adventure               int64
Fantasy                 int64
IMAX                    int64
Animation               int64
Musical                 int64
Horror                  int64
Film-Noir               int64
Western                 int64
Mystery                 int64
(no genres listed)      int64
dtype: object

In [16]:
# Assuming your DataFrame is named df

columns_not_genres = ['movieId', 'title', 'year', 'userId', 'rating', 'date', '(no genres listed)']
for column in df.columns:
    if column not in columns_not_genres :
        df.rename(columns={column: 'genre_' + column}, inplace=True)


In [17]:
# We print the columns of the dataframe
df.columns

Index(['movieId', 'title', 'year', 'userId', 'rating', 'date', 'genre_Drama',
       'genre_Action', 'genre_Sci-Fi', 'genre_Comedy', 'genre_Crime',
       'genre_Thriller', 'genre_Romance', 'genre_War', 'genre_Documentary',
       'genre_Children', 'genre_Adventure', 'genre_Fantasy', 'genre_IMAX',
       'genre_Animation', 'genre_Musical', 'genre_Horror', 'genre_Film-Noir',
       'genre_Western', 'genre_Mystery', '(no genres listed)'],
      dtype='object')

# Item-based Rating Prediction (ItemKNN)

### Step 1: User-Item matrix construction
The first thing we have to do is build the user-item matrix:

- Choose a similarity metric to calculate the similarity between items. Common metrics include cosine similarity, Pearson correlation coefficient, and Jaccard similarity.
- Calculate the similarity between each pair of items based on the ratings provided by users. This will result in an item-item similarity matrix.

##### Libraries Used
- **pandas**: A powerful data manipulation library in Python.
- **numpy**: A library for numerical computing in Python.
- **sklearn.metrics.pairwise.cosine_similarity**: A function from scikit-learn used to compute the cosine similarity between vectors.

In [18]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame with the specified format
# Convert userId column to integer type if needed
if df['userId'].dtype != int:
    df['userId'] = df['userId'].astype(int)

# Get all unique user IDs and movie IDs
all_user_ids = np.unique(df['userId'])
all_movie_ids = np.unique(df['movieId'])

# Create a DataFrame with all combinations of user IDs and movie IDs
all_user_movie_pairs = np.array(np.meshgrid(all_user_ids, all_movie_ids)).T.reshape(-1, 2)
df_pairs = pd.DataFrame(all_user_movie_pairs, columns=['userId', 'movieId'])

# Merge the original DataFrame with all_user_movie_pairs to fill missing ratings with 0
df_filled = pd.merge(df_pairs, df, on=['userId', 'movieId'], how='left').fillna(0)

# Create user-item matrix using pivot_table
item_user_matrix = pd.pivot_table(df_filled, values='rating', index='userId', columns='movieId', fill_value=0)

# Convert user-item matrix to NumPy array for faster computation
item_user_array = item_user_matrix.to_numpy()


In [19]:
item_user_matrix.head()

movieId,1,4,15,30,43,89,104,108,122,146,...,174479,174551,175475,176371,176389,177593,179813,181413,185029,186587
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
item_user_array

array([[4. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [2.5, 0. , 0. , ..., 0. , 0. , 0. ],
       [3. , 0. , 0. , ..., 0. , 0. , 0. ],
       [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

### Train-test split

The train-val-test split is a technique used in machine learning to evaluate the performance of a model. It involves dividing the dataset into three subsets: the training set, the validation set, and the test set.

The training set is used to train the model and optimize its parameters.
The validation set is used to fine-tune the model and select the best hyperparameters.
The test set is used to evaluate the final performance of the model on unseen data.
By using a train-val-test split, we can assess the model's performance on unseen data and ensure that it generalizes well to new examples. It helps prevent overfitting and provides a more reliable estimate of the model's performance in real-world scenarios.

In [21]:
from sklearn.model_selection import train_test_split
# Split the data into training and test sets
train_val, test = train_test_split(item_user_array, test_size=0.2, random_state=42)

# Split the training set into training and validation sets
train, val = train_test_split(train_val, test_size=0.2, random_state=42)

# Print the shapes of the datasets
print("Training set shape:", train.shape)
print("Validation set shape:", val.shape)
print("Test set shape:", test.shape)

# We are also going to do the split for the matrix df
# Split the user-item matrix into training and test sets
train_val_matrix, test_matrix = train_test_split(item_user_matrix, test_size=0.2, random_state=42)

# Split the training set matrix into training and validation sets
train_matrix, val_matrix = train_test_split(train_val_matrix, test_size=0.2, random_state=42)

# We print a ' ' to give some space inbetween lines
print(' ')

# Print the shapes of the matrix datasets
print("Training set matrix shape:", train_matrix.shape)
print("Validation set matrix shape:", val_matrix.shape)
print("Test set matrix shape:", test_matrix.shape)


Training set shape: (340, 487)
Validation set shape: (85, 487)
Test set shape: (107, 487)
 
Training set matrix shape: (340, 487)
Validation set matrix shape: (85, 487)
Test set matrix shape: (107, 487)


In [22]:
train_matrix

movieId,1,4,15,30,43,89,104,108,122,146,...,174479,174551,175475,176371,176389,177593,179813,181413,185029,186587
userId,,,,,,,,,,,,,,,,,,,,,
428,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
517,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
train

array([[0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
# Calculate cosine similarity between items using NumPy functions
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

# Calculate item-item similarity matrix for train data
#item_similarity_matrix_train = np.zeros((train.shape[1], train.shape[1]))

#for i in range(train.shape[1]):
 #   for j in range(i, train.shape[1]):
  #      item_similarity_matrix_train[i, j] = cosine_similarity(train[:, i], train[:, j])
   #     item_similarity_matrix_train[j, i] = item_similarity_matrix_train[i, j]

# Adjusting the mapping to start indexing from 0
user_id_to_index = {user_id: i for i, user_id in enumerate(item_user_matrix.index)}
index_to_user_id = {i: user_id for i, user_id in enumerate(item_user_matrix.index)}

# Create a mapping from movie IDs to indices
movie_id_to_index = {movie_id: i for i, movie_id in enumerate(item_user_matrix.columns)}
index_to_movie_id = {i: movie_id for i, movie_id in enumerate(item_user_matrix.columns)}




### Similarity Calculation utilizing Genres:



In [38]:
import pandas as pd

# Step 1: Create item-genre matrix
def create_item_genre_matrix(df, train):
    # Extract genre columns from the dataframe
    genre_columns = [col for col in df.columns if col.startswith('genre_')]
    
    # Initialize item-genre matrix with zeros
    item_genre_matrix = pd.DataFrame(0, index=genre_columns, columns=train.columns)
    
    # Fill the matrix with genre information
    for i, row in df.iterrows():
        
        item_id = row['movieId']  # Assuming 'movieId' is the column containing item IDs
        genres = row[genre_columns].values
        item_genre_matrix[item_id] = genres
    
    return item_genre_matrix

# Example usage:
# Assuming df is your original dataframe with genre information and train_matrix is your train set dataframe
item_genre_matrix = create_item_genre_matrix(df, train_matrix)

# Example usage:
print(item_genre_matrix)


movieId           1      4      15     30     43     89     104    108     \
genre_Drama            0      1      0      1      1      0      0      0   
genre_Action           0      0      1      0      0      1      0      0   
genre_Sci-Fi           0      0      0      0      0      0      0      0   
genre_Comedy           1      1      0      0      0      0      1      0   
genre_Crime            0      0      0      1      0      0      0      0   
genre_Thriller         0      0      0      0      0      1      0      0   
genre_Romance          0      1      1      0      0      0      0      0   
genre_War              0      0      0      0      0      0      0      0   
genre_Documentary      0      0      0      0      0      0      0      1   
genre_Children         1      0      0      0      0      0      0      0   
genre_Adventure        1      0      1      0      0      0      0      0   
genre_Fantasy          1      0      0      0      0      0      0      0   

In [29]:
item_genre_matrix

item_genre_matrix.shape

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

(487, 19)

#### Collaborative Filtering with Cosine Similarity

This code snippet demonstrates how to perform item-based collaborative filtering using cosine similarity. Collaborative filtering is a technique commonly used in recommendation systems to predict a user's preferences for items based on the preferences of similar users/items.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def compute_cosine_similarity(train, dfoak):
    # Extract the genre columns from df
    genre_columns = df.columns[6:]

    # Compute the cosine similarity between the genre columns in df and the train matrix
    similarity_matrix = cosine_similarity(df[genre_columns], train)

    return similarity_matrix


In [ ]:
item_similarity_matrix_train

array([[1.]])

In [ ]:
movie_id_to_index

{1: 0,
 4: 1,
 15: 2,
 30: 3,
 43: 4,
 89: 5,
 104: 6,
 108: 7,
 122: 8,
 146: 9,
 290: 10,
 303: 11,
 305: 12,
 346: 13,
 353: 14,
 358: 15,
 363: 16,
 389: 17,
 393: 18,
 416: 19,
 423: 20,
 478: 21,
 504: 22,
 522: 23,
 551: 24,
 556: 25,
 605: 26,
 627: 27,
 640: 28,
 694: 29,
 708: 30,
 728: 31,
 790: 32,
 810: 33,
 835: 34,
 901: 35,
 953: 36,
 1003: 37,
 1006: 38,
 1037: 39,
 1040: 40,
 1046: 41,
 1207: 42,
 1261: 43,
 1266: 44,
 1271: 45,
 1272: 46,
 1358: 47,
 1373: 48,
 1388: 49,
 1391: 50,
 1398: 51,
 1415: 52,
 1432: 53,
 1446: 54,
 1447: 55,
 1507: 56,
 1518: 57,
 1564: 58,
 1572: 59,
 1580: 60,
 1597: 61,
 1625: 62,
 1665: 63,
 1757: 64,
 1769: 65,
 1816: 66,
 1882: 67,
 1940: 68,
 1955: 69,
 1964: 70,
 1969: 71,
 1995: 72,
 2004: 73,
 2013: 74,
 2037: 75,
 2046: 76,
 2071: 77,
 2107: 78,
 2114: 79,
 2148: 80,
 2160: 81,
 2171: 82,
 2177: 83,
 2273: 84,
 2302: 85,
 2310: 86,
 2320: 87,
 2324: 88,
 2325: 89,
 2336: 90,
 2344: 91,
 2356: 92,
 2358: 93,
 2366: 94,
 2378: 95,

In [ ]:
print(index_to_user_id)

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 9, 8: 10, 9: 11, 10: 12, 11: 13, 12: 14, 13: 15, 14: 16, 15: 17, 16: 18, 17: 19, 18: 20, 19: 21, 20: 22, 21: 23, 22: 24, 23: 25, 24: 27, 25: 28, 26: 29, 27: 30, 28: 31, 29: 32, 30: 33, 31: 34, 32: 36, 33: 38, 34: 39, 35: 40, 36: 41, 37: 42, 38: 43, 39: 44, 40: 45, 41: 46, 42: 47, 43: 48, 44: 50, 45: 51, 46: 52, 47: 54, 48: 57, 49: 58, 50: 59, 51: 62, 52: 63, 53: 64, 54: 65, 55: 66, 56: 67, 57: 68, 58: 69, 59: 70, 60: 71, 61: 73, 62: 74, 63: 75, 64: 76, 65: 78, 66: 79, 67: 80, 68: 82, 69: 83, 70: 84, 71: 85, 72: 86, 73: 87, 74: 88, 75: 89, 76: 90, 77: 91, 78: 92, 79: 93, 80: 95, 81: 96, 82: 97, 83: 98, 84: 99, 85: 100, 86: 101, 87: 103, 88: 104, 89: 105, 90: 106, 91: 107, 92: 109, 93: 110, 94: 111, 95: 112, 96: 113, 97: 114, 98: 115, 99: 116, 100: 117, 101: 118, 102: 119, 103: 121, 104: 122, 105: 123, 106: 124, 107: 125, 108: 129, 109: 130, 110: 131, 111: 132, 112: 134, 113: 135, 114: 136, 115: 137, 116: 138, 117: 139, 118: 140, 119: 141, 12

In [ ]:
print(user_id_to_index)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 9: 7, 10: 8, 11: 9, 12: 10, 13: 11, 14: 12, 15: 13, 16: 14, 17: 15, 18: 16, 19: 17, 20: 18, 21: 19, 22: 20, 23: 21, 24: 22, 25: 23, 27: 24, 28: 25, 29: 26, 30: 27, 31: 28, 32: 29, 33: 30, 34: 31, 36: 32, 38: 33, 39: 34, 40: 35, 41: 36, 42: 37, 43: 38, 44: 39, 45: 40, 46: 41, 47: 42, 48: 43, 50: 44, 51: 45, 52: 46, 54: 47, 57: 48, 58: 49, 59: 50, 62: 51, 63: 52, 64: 53, 65: 54, 66: 55, 67: 56, 68: 57, 69: 58, 70: 59, 71: 60, 73: 61, 74: 62, 75: 63, 76: 64, 78: 65, 79: 66, 80: 67, 82: 68, 83: 69, 84: 70, 85: 71, 86: 72, 87: 73, 88: 74, 89: 75, 90: 76, 91: 77, 92: 78, 93: 79, 95: 80, 96: 81, 97: 82, 98: 83, 99: 84, 100: 85, 101: 86, 103: 87, 104: 88, 105: 89, 106: 90, 107: 91, 109: 92, 110: 93, 111: 94, 112: 95, 113: 96, 114: 97, 115: 98, 116: 99, 117: 100, 118: 101, 119: 102, 121: 103, 122: 104, 123: 105, 124: 106, 125: 107, 129: 108, 130: 109, 131: 110, 132: 111, 134: 112, 135: 113, 136: 114, 137: 115, 138: 116, 139: 117, 140: 118, 141: 119, 14

In [ ]:
item_user_array.shape

(532, 487)

In [ ]:
item_user_matrix.shape

(532, 487)

In [ ]:
# We print the item_similarity_matrix
item_similarity_matrix_train

array([[1.        , 1.        , 0.54772256, ..., 0.96336399, 0.96896279,
        0.96896279],
       [1.        , 1.        , 0.54772256, ..., 0.96336399, 0.96896279,
        0.96896279],
       [0.54772256, 0.54772256, 1.        , ..., 0.53530338, 0.54433105,
        0.54433105],
       ...,
       [0.96336399, 0.96336399, 0.53530338, ..., 1.        , 0.99902487,
        0.99902487],
       [0.96896279, 0.96896279, 0.54433105, ..., 0.99902487, 1.        ,
        1.        ],
       [0.96896279, 0.96896279, 0.54433105, ..., 0.99902487, 1.        ,
        1.        ]])

In [ ]:
item_similarity_matrix_train.shape

(4665, 4665)

### Step 2: Neighborhood Selection
- Determine the neighborhood size, i.e., the number of most similar items to consider when predicting ratings for a target item.
- Select the most similar items for each item in your dataset based on their calculated similarities. This forms the neighborhood for each item.

#### Item-Based Neighborhoods and Ratings Aggregation

This code following snippet enhances the previous item-based collaborative filtering approach by considering ratings aggregation within the item neighborhoods.

##### Steps:

1. **Defining Neighborhood Size**:
   - The variable `neighborhood_size` determines the number of most similar items to consider in the neighborhood.

2. **Initializing Data Structure**:
   - An empty dictionary `item_neighborhoods` is initialized to store the neighborhoods for each item.

3. **Iterating Over Items**:
   - For each movie in the dataset:
     - All ratings for the current movie are extracted from the DataFrame (`df`).
     - Ratings aggregation is performed. In this example, the average rating for the movie is computed, but other aggregation methods can be used.
     - The similarity scores for the current movie are retrieved from the precomputed `item_similarity_matrix`.
     - Similarity scores are sorted in descending order, and the indices of the most similar items (excluding itself) are obtained.
     - These indices are converted back to movie IDs, forming the neighborhood for the current item.
     - The neighborhood for the current item is stored in the `item_neighborhoods` dictionary.

4. **Output**:
   - `item_neighborhoods`: A dictionary where keys are movie IDs, and values are lists of movie IDs representing the neighborhood of each item. Each movie's neighborhood includes movies with similar ratings and content.

##### Note:
- This approach considers both similarity in ratings and content (as captured by cosine similarity) when building item neighborhoods.
- Aggregating ratings within item neighborhoods helps in providing more personalized recommendations.
- The choice of rating aggregation method (e.g., mean, median) can impact the quality of recommendations and may need to be adjusted based on the characteristics of the dataset and user preferences.


In [ ]:
import numpy as np

def neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix, neighborhood_size=5):
    # Initialize an empty dictionary to store item neighborhoods
    item_neighborhoods = {}

    # Iterate over each item (movie) index in the dataset
    for movie_index in range(item_user_array.shape[1]):
        # Convert the item index to movie ID
        movie_id = index_to_movie_id[movie_index]

        # Extract all ratings for the current movie
        movie_ratings = item_user_array[:, movie_index]

        # Aggregate ratings (e.g., compute the mean rating)
        movie_avg_rating = np.mean(movie_ratings)

        # Retrieve similarity scores for the current movie
        similarity_scores = item_similarity_matrix[movie_index]

        # Sort similarity scores in descending order and get indices of most similar items
        most_similar_indices = np.argsort(similarity_scores)[::-1][1:neighborhood_size+1]

        # Convert indices back to movie IDs to form the neighborhood
        neighborhood = [index_to_movie_id[idx] for idx in most_similar_indices]

        # Store the neighborhood for the current item in the item_neighborhoods dictionary
        item_neighborhoods[movie_id] = neighborhood

    return item_neighborhoods

# Example usage:
item_neighborhoods_train = neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix_train)
print(item_neighborhoods_train)


KeyError: 2893

### Step 4: Rating Prediction

- For each target item and user pair where the user hasn't rated the target item:
Identify the neighborhood of similar items to the target item.
- Predict the rating for the target item using a weighted average of the ratings of the items in its neighborhood, where the weights are the similarities between the items and the target item.
- Adjust the prediction based on the user's average rating or other normalization techniques, if necessary.

In [ ]:
# Train your model and create the item_neighborhoods dictionary using the training data (steps 1-3)

# Initialize an empty DataFrame to store predicted ratings
predicted_ratings_train = pd.DataFrame(index=train_matrix.index, columns=train_matrix.columns)

# Iterate over each user-item pair in the training set
for user_id, user_ratings in train_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        
        # Skip if the rating is non-zero (indicating a rating given by the user)
        if rating != 0:
            continue
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train:
            neighborhood = item_neighborhoods_train[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the training set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_train.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_train.fillna(predicted_ratings_train.mean().mean(), inplace=True)

# Display the head of the predicted_ratings DataFrame
predicted_ratings_train.head()


KeyboardInterrupt: 

In [ ]:
# We count the NaN values in the predicted_ratings_train dataframe
predicted_ratings_train.isnull().sum().sum()

0

In [ ]:
def get_top_recommendations(user_id, predicted_ratings, df):
    """
    Get top movie recommendations for a given user using predicted ratings.

    Parameters:
        user_id (int): ID of the user for whom recommendations are to be generated.
        predicted_ratings (pd.DataFrame): DataFrame containing predicted ratings for users and movies.
        df (pd.DataFrame): DataFrame containing movie ratings.

    Returns:
        list: Top movie titles recommended for the user.
    """
    # Ensure that predicted_ratings is a DataFrame
    if not isinstance(predicted_ratings, pd.DataFrame):
        raise ValueError("predicted_ratings must be a pandas DataFrame")

    # Check if the user ID exists in the predicted ratings DataFrame's index
    if user_id not in predicted_ratings.index:
        # If the user ID doesn't exist, return an empty list
        return ["The user doesn't exist"]

    # Get the predicted ratings for the user
    user_predicted_ratings = predicted_ratings.loc[user_id]
    
    # Filter out the movies that the user has already seen
    seen_movies = set(df[df['userId'] == user_id]['movieId'])
    unseen_movies = [movie_id for movie_id in user_predicted_ratings.index if movie_id not in seen_movies]
    
    # Check if there are unseen movies with predicted ratings
    if not unseen_movies:
        # If all movies have been seen, return an empty list
        return []
    
    # Sort the unseen movies by predicted rating in descending order
    sorted_unseen_movies = user_predicted_ratings[unseen_movies].sort_values(ascending=False)
    
    # Get the top 5 movie titles
    top_movie_ids = sorted_unseen_movies.head(5).index.tolist()
    
    # Get the unique movie titles corresponding to the top movie IDs
    top_movie_titles = set(df[df['movieId'].isin(top_movie_ids)]['title'])
    
    return list(top_movie_titles)[:5]  # Return only the first 5 unique movie titles


In [ ]:
top_recommendations = []

# Iterate over each row (user) in the train_matrix
for user_id in train_matrix.index:
    # Get the actual user ID corresponding to the user index
    recommendations = get_top_recommendations(user_id, predicted_ratings_train, df)
    top_recommendations.append(recommendations)

# Output: List of top movie recommendations for each user in the train matrix
top_recommendations

# print size of top_recommendations
len(top_recommendations)


[['Toy Story ',
  'Vie en Rose, La (Môme, La) ',
  'How the Grinch Stole Christmas! ',
  'Disturbia ',
  'Sunshine '],
 ['Toy Story ',
  'Grindhouse ',
  'Hoax, The ',
  'Black Book (Zwartboek) ',
  'Sunshine '],
 ['Toy Story ',
  'Vacancy ',
  'In the Land of Women ',
  'Vie en Rose, La (Môme, La) ',
  "It's a Very Merry Muppet Christmas Movie "],
 ['Toy Story ',
  'Vacancy ',
  'In the Land of Women ',
  'Vie en Rose, La (Môme, La) ',
  "It's a Very Merry Muppet Christmas Movie "],
 ['Toy Story ',
  'Vie en Rose, La (Môme, La) ',
  'Fracture ',
  'Hoax, The ',
  'Disturbia '],
 ['Room for Romeo Brass, A ', 'Creep ', 'Toy Story ', 'Rent ', 'Wolf Creek '],
 ['Vie en Rose, La (Môme, La) ',
  'Fracture ',
  'Hoax, The ',
  'Waiting to Exhale ',
  'Disturbia '],
 ['21 Up ',
  'In the Land of Women ',
  'Maniac Cop ',
  'Sharkwater ',
  'Waiting to Exhale '],
 ['Toy Story ',
  'Ugly Duckling and Me!, The ',
  'Fracture ',
  'Because I Said So ',
  'Astronaut Farmer, The '],
 ['Toy Story ',

389

In [ ]:
print("Last version")

Last version


### Step 5: Model Evaluation
- We evaluate the performance of your ItemKNN algorithm using appropriate evaluation metrics such as Root Mean Squared Error (RMSE), Mean Absolute Error (MAE), or others.
- Split your dataset into training and testing sets to assess the model's predictive accuracy on unseen data.

In [ ]:
train.shape

predicted_ratings_train.shape

(389, 2923)

(389, 2923)

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate_model(train_matrix, predicted_ratings):
    """
    Evaluate the model's performance on the training data.

    Parameters:
        train_matrix (numpy.ndarray): Item-user matrix from the training data.
        predicted_ratings (pandas.DataFrame or numpy.ndarray): Predicted ratings DataFrame or array for the training data.

    Returns:
        float: Mean Absolute Error (MAE) on the training data.
        float: Root Mean Squared Error (RMSE) on the training data.
    """
    # Convert predicted_ratings to a numpy array if it's a DataFrame
    if isinstance(predicted_ratings, pd.DataFrame):
        predicted_ratings = predicted_ratings.to_numpy()

    # Ensure train_matrix and predicted_ratings have the same shape
    assert train_matrix.shape == predicted_ratings.shape, "Shapes of train_matrix and predicted_ratings are not consistent."

    # Initialize lists to store true and predicted ratings
    true_ratings = []
    pred_ratings = []

    # Iterate over each user and their ratings
    for user_id, user_ratings in enumerate(train_matrix):
        for movie_id, rating in enumerate(user_ratings):
            # Skip unrated movies
            if rating == 0:
                continue

            # Check if the indices are within the bounds of the predicted ratings array
            if user_id < predicted_ratings.shape[0] and movie_id < predicted_ratings.shape[1]:
                # Get the predicted rating for the corresponding movie
                pred_rating = predicted_ratings[user_id, movie_id]

                # Append the true and predicted ratings
                true_ratings.append(rating)
                pred_ratings.append(pred_rating)

    # Convert lists to numpy arrays
    true_ratings = np.array(true_ratings)
    pred_ratings = np.array(pred_ratings)

    # Calculate evaluation metrics
    mae = mean_absolute_error(true_ratings, pred_ratings)
    rmse = np.sqrt(mean_squared_error(true_ratings, pred_ratings))

    return mae, rmse

# Assuming train is the numpy array representing the item-user matrix
# Assuming predicted_ratings_train is the DataFrame or numpy array representing the predicted ratings for the training data

# Evaluate the model
mae, rmse = evaluate_model(train, predicted_ratings_train)

print("Mean Absolute Error (MAE) on Training Data:", mae)
print("Root Mean Squared Error (RMSE) on Training Data:", rmse)


Mean Absolute Error (MAE) on Training Data: 0.8206720311290793
Root Mean Squared Error (RMSE) on Training Data: 1.0387696367207275


### Step 6: Parameter Tuning
- Experiment with different parameters such as similarity threshold, neighborhood size, and similarity metric to optimize the performance of your ItemKNN algorithm.
- Use cross-validation or other techniques to tune these parameters and avoid overfitting.

1. **Import Necessary Libraries:**
   - We import the required libraries for performing grid search cross-validation (`GridSearchCV`), creating custom scorers (`make_scorer`), and utilizing the `NearestNeighbors` algorithm.

2. **Define Cosine Similarity Function:**
   - We define a custom function `cosine_similarity` to compute the cosine similarity between two vectors. This function calculates the dot product of the vectors and divides it by the product of their norms.

3. **Define Custom Scorer:**
   - We create a custom scorer `cosine_similarity_scorer` using `make_scorer`, which enables us to use cosine similarity as the scoring metric during grid search cross-validation.

4. **Define Parameter Grid:**
   - We specify a parameter grid `param_grid` containing the hyperparameters to be tuned. In this case, we're tuning the number of neighbors (`n_neighbors`) and the distance metric (`metric`) for the `NearestNeighbors` algorithm.

5. **Initialize NearestNeighbors Model:**
   - We initialize the `NearestNeighbors` model without specifying any hyperparameters.

6. **Create GridSearchCV Object:**
   - We create a `GridSearchCV` object named `grid_search` with the specified parameter grid, cross-validation strategy (5-fold cross-validation), and custom scoring metric (`cosine_similarity_scorer`).

7. **Fit the Data:**
   - We fit the `item_user_matrix` data to the `grid_search` object to perform hyperparameter tuning. `item_user_matrix` typically contains the item-item similarity matrix computed using collaborative filtering techniques.

8. **Get Best Hyperparameters:**
   - After fitting the data, we retrieve the best hyperparameters selected by the grid search using the `best_params_` attribute of the `grid_search` object.

9. **Print Best Parameters:**
   - Finally, we print the best hyperparameters obtained from the grid search.



`scikit-learn (sklearn)`:

- Scikit-learn is a popular machine learning library in Python that provides simple and efficient tools for data analysis and modeling.
- It includes various modules for tasks such as classification, regression, clustering, dimensionality reduction, and model selection.
- The GridSearchCV class from scikit-learn is used for hyperparameter tuning through grid search along with cross-validation.
- The make_scorer function allows you to create a custom scoring function for use with GridSearchCV.
- The NearestNeighbors class provides functionality for unsupervised nearest neighbors learning, which can be used for tasks such as finding k-nearest neighbors for a given data point.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import make_scorer
import numpy as np

# Define a custom scorer based on cosine similarity defined above
cosine_similarity_scorer = make_scorer(cosine_similarity)

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 30, 40],
    'metric': ['cosine', 'euclidean']
}

# Initialize NearestNeighbors model
knn_model = NearestNeighbors()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring=cosine_similarity_scorer)

# Fit the data to perform hyperparameter tuning
grid_search.fit(train_matrix)  # train_matrix contains item-user matrix for the train set

# Get the best hyperparameters
best_params = grid_search.best_params_

print("Best parameters:", best_params)


c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

GridSearchCV(cv=5, estimator=NearestNeighbors(),
             param_grid={'metric': ['cosine', 'euclidean'],
                         'n_neighbors': [5, 10, 15, 30, 40]},
             scoring=make_scorer(cosine_similarity, response_method='predict'))

Best parameters: {'metric': 'cosine', 'n_neighbors': 5}


This code sets up a GridSearchCV object to perform hyperparameter tuning using the train set (train_matrix). It explores different combinations of hyperparameters specified in the param_grid, evaluates them using 5-fold cross-validation (cv=5), and uses the cosine_similarity scorer to optimize the model's performance based on cosine similarity. Finally, it prints the best hyperparameters found during the search.

We have to perform the calculation of the predicted ratings for the train-val set to generate predicted ratings in order to evaluate the performance of our trained model on a validation dataset. 

If we computed the similarity matrix again specifically for the train_val set, it would essentially mean that we are using a different set of similarity measures for predicting ratings compared to what we used during training. This approach could lead to inconsistencies and potentially degrade the performance of our model. Here's what could happen:

1. **Inconsistency**: The similarity measures computed for the train_val set might differ from those computed for the training set due to variations in the data. As a result, the predicted ratings based on these new similarity measures may not align well with the predictions made during training, leading to inconsistency in the model's behavior.

2. **Overfitting**: Computing a new similarity matrix specifically for the train_val set might lead to overfitting on the validation data. The model may capture noise or idiosyncrasies present in the train_val set, which may not generalize well to unseen data.

3. **Increased Complexity**: Computing the similarity matrix again for the train_val set adds computational complexity and redundancy, especially if the similarity computation process is resource-intensive. This can result in longer training times and increased resource utilization.

Overall, it's generally recommended to use the same similarity measures or neighborhood definitions for both training and validation sets to ensure consistency and generalizability of the model.

In [ ]:
# Assuming you have already trained your model and obtained the item_neighborhoods_train dictionary

# Initialize an empty DataFrame to store predicted ratings for train_val
predicted_ratings_val = pd.DataFrame(index=val_matrix.index, columns=val_matrix.columns)

# Iterate over each user and their ratings in the train_val set
for user_id, user_ratings in val_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        # Skip if the rating is non-zero (indicating a rating given by the user)
        if rating != 0:
            continue
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train:
            neighborhood = item_neighborhoods_train[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the train set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_val.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_val.fillna(predicted_ratings_val.mean().mean(), inplace=True)


C:\Users\Jaume\AppData\Local\Temp\ipykernel_8932\2864728636.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings_val.fillna(predicted_ratings_val.mean().mean(), inplace=True)


In [ ]:
predicted_ratings_val.head()

predicted_ratings_val.shape

movieId,1,4,9,11,13,15,18,20,24,30,...,185473,186587,187031,187541,188675,188797,191005,193573,193583,193587
userId,,,,,,,,,,,,,,,,,,,,,
270,3.634821,3.625000,3.634821,3.625000,3.625000,3.625000,3.625000,3.625000,3.625000,3.625000,...,3.625000,3.625000,3.625000,3.625000,3.625000,3.625000,3.625000,3.625000,3.625000,3.625000
124,3.634821,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250,...,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250
368,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,...,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224
213,3.634821,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,...,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667
63,3.634821,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,...,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423


(98, 2923)

Now we evaluate the model with the Validation item-user matrix.

In [ ]:
# Now, you can evaluate the model using the evaluate_model function
mae, rmse = evaluate_model(val_matrix.to_numpy(), predicted_ratings_val)

print("Mean Absolute Error (MAE) on Validation Data:", mae)
print("Root Mean Squared Error (RMSE) on Validation Data:", rmse)


Mean Absolute Error (MAE) on Validation Data: 0.8302728368954743
Root Mean Squared Error (RMSE) on Validation Data: 1.0484103421666116


### Retraining of the model with the Train-Val set

In [ ]:
# After the model evaluation on the validation set, we recalculate the similarity matrix
# of the model on the train-val data
item_similarity_matrix_train_val = calculate_item_similarity_matrix(train_val)

# We calculate the item neighborhoods for the train_val data 
item_neighborhoods_train_val = neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix_train_val)

C:\Users\Jaume\AppData\Local\Temp\ipykernel_8932\2934347206.py:6: RuntimeWarning: invalid value encountered in scalar divide
  similarity = dot_product / (norm_a * norm_b)


### Step 7: Deployment

Once we have come up with the best parameters possible and trained the model with the whole train_validation set, we will test it on the test set.

In [ ]:
test_matrix

movieId,1,4,9,11,13,15,18,20,24,30,...,185473,186587,187031,187541,188675,188797,191005,193573,193583,193587
userId,,,,,,,,,,,,,,,,,,,,,
293,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
367,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
469,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
119,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Check if there are NaN values in the test matrix
test_matrix.isnull().sum().sum()

0

In [ ]:
# Now, we can evaluate the model's predicted ratings on the test set

# Initialize an empty DataFrame to store predicted ratings
predicted_ratings_test = pd.DataFrame(index=test_matrix.index, columns=test_matrix.columns)

# Iterate over each user-item pair in the training set
for user_id, user_ratings in test_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        
        # Skip if the rating is non-zero (indicating a rating given by the user)
        if rating != 0:
            continue
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train_val:
            neighborhood = item_neighborhoods_train_val[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the training set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_test.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_test.fillna(predicted_ratings_test.mean().mean(), inplace=True)

# Now, you can evaluate the model's predicted ratings on the test set
mae, rmse = evaluate_model(test_matrix.to_numpy(), predicted_ratings_test.to_numpy())

print("Mean Absolute Error (MAE) on Test Data using the Ratings given to the Test Set:", mae)
print("Root Mean Squared Error (RMSE) on Test Data using the Ratings given to the Test Set:", rmse)


Mean Absolute Error (MAE) on Test Data using Predicted Ratings from Test Set: 0.880777353578179
Root Mean Squared Error (RMSE) on Test Data using Predicted Ratings from Test Set: 1.119299084054372


C:\Users\Jaume\AppData\Local\Temp\ipykernel_8932\912451769.py:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings_test.fillna(predicted_ratings_test.mean().mean(), inplace=True)


In [ ]:
top_recommendations_test = []

# Iterate over each row (user) in the test_matrix
for user_id in test_matrix.index:
    # Get the actual user ID corresponding to the user index
    recommendations = get_top_recommendations(user_id, predicted_ratings_test, df)
    top_recommendations_test.append(recommendations)

# Output: List of top movie recommendations for each user in the test matrix
top_recommendations_test

# print size of top_recommendations
print("Size of top_recommendations_test:", len(top_recommendations_test))



[['Vie en Rose, La (Môme, La) ',
  'Waiting to Exhale ',
  'How the Grinch Stole Christmas! ',
  'Disturbia ',
  'Sunshine '],
 ["It's My Party ",
  'Warriors, The ',
  'Prince & Me, The ',
  'Always Watching: A Marble Hornets Story ',
  'Volcano '],
 ['Commando ',
  'Caligula ',
  'Tremors II: Aftershocks ',
  'King and I, The ',
  'Dawn of the Dead '],
 ['Toy Story ',
  'Vie en Rose, La (Môme, La) ',
  'How the Grinch Stole Christmas! ',
  'Disturbia ',
  'Sunshine '],
 ['Hurlyburly ',
  'Genghis Blues ',
  'Family Thing, A ',
  'I Walked with a Zombie ',
  'Paul '],
 ['Toy Story ',
  'Vacancy ',
  'In the Land of Women ',
  'Fracture ',
  'Disturbia '],
 ['Toy Story ',
  '21 Up ',
  'In the Land of Women ',
  'Maniac Cop ',
  'Sharkwater '],
 ['Toy Story ',
  "Block Party (a.k.a. Dave Chappelle's Block Party) ",
  'Final Destination 3 ',
  'Something New ',
  'American Haunting, An '],
 ['Maniac Cop ',
  "It's a Very Merry Muppet Christmas Movie ",
  'Sharkwater ',
  'Day Watch (Dne

Size of top_recommendations_test: 122


# Item-based Classification (ItemKNN)

### Step 1: Data preparation
- Ensure you have a dataset with user-item interactions. Each interaction should include the user ID, item ID, and the corresponding label or class for classification.
- Preprocess the data as needed, including handling missing values, encoding categorical variables, and splitting into training and testing sets.

For this step we are going to resuse the `item_user_matrix` that we created in the first model built. 

### Step 2: Compute Item Similarity


In [ ]:
item_similarity_matrix_train.shape

(2923, 2923)

In [ ]:
import numpy as np

def item_neighborhood_selection(similarity_matrix, k=None, threshold=None, item_ids=None):
    """
    Select a subset of similar items for each item based on similarity matrix.

    Parameters:
        similarity_matrix (numpy.ndarray): Item-item similarity matrix.
        k (int): Number of similar items to select (optional).
        threshold (float): Similarity threshold for selecting similar items (optional).
        item_ids (list): List of item IDs corresponding to rows/columns of the similarity matrix.

    Returns:
        dict: Dictionary containing similar items for each item.
    """
    num_items = similarity_matrix.shape[0]
    item_neighborhood = {}

    for i in range(num_items):
        if k is not None:
            # Select top-k similar items (excluding the item itself)
            similar_items_indices = np.argsort(similarity_matrix[i])[::-1][:k]
        elif threshold is not None:
            # Select items with similarity above threshold (excluding the item itself)
            similar_items_indices = np.where(similarity_matrix[i] > threshold)[0]

        # Remove the item itself from the neighborhood
        similar_items_indices = similar_items_indices[similar_items_indices != i]

        # Get the item ID corresponding to the current index
        current_item_id = item_ids[i] if item_ids is not None else i

        # Get the item IDs for the neighborhood
        neighborhood_item_ids = [item_ids[index] for index in similar_items_indices]

        # Store the similar items with the item's ID as the key
        item_neighborhood[current_item_id] = neighborhood_item_ids

    return item_neighborhood

# Example usage:
# Assuming you have an item-item similarity matrix 'item_similarity_matrix_train'
# and you want to select top-5 similar items for each item
k = 5
item_neighborhoods_classification_train = item_neighborhood_selection(item_similarity_matrix_train, k=k, item_ids=index_to_movie_id)

# Check if there are any items with empty neighborhoods
empty_neighborhoods = []

for item_id, neighborhood in item_neighborhoods_classification_train.items():
    if not neighborhood:
        empty_neighborhoods.append(item_id)

if empty_neighborhoods:
    print("Items with empty neighborhoods:", empty_neighborhoods)
else:
    print("No items with empty neighborhoods found.")


No items with empty neighborhoods found.


In [ ]:
len(item_neighborhoods_classification_train)

2923

### Step 3: Ratings classification selection

In [ ]:
import numpy as np
import pandas as pd

def recommend_movies(train_matrix, item_neighborhoods, movie_id_to_index, num_movies=5):
    """
    Recommend top movies for each user based on item neighborhood.

    Parameters:
        train_matrix (np.ndarray or pd.DataFrame): Matrix containing user-item ratings in the train set.
        item_neighborhoods (dict): Dictionary containing similar items for each item.
        movie_id_to_index (dict): Dictionary mapping movie IDs to their corresponding indices in the train_matrix.
        num_movies (int): Number of movies to recommend for each user.

    Returns:
        pd.DataFrame: DataFrame containing predicted ratings for recommended movies for each user.
    """
    # Convert train_matrix to DataFrame if it's a numpy array
    if isinstance(train_matrix, np.ndarray):
        train_matrix = pd.DataFrame(train_matrix, index=np.arange(train_matrix.shape[0]), columns=np.arange(train_matrix.shape[1]))

    # Initialize an empty DataFrame to store predicted ratings
    predicted_ratings = pd.DataFrame(index=train_matrix.index, columns=train_matrix.columns)

    # Iterate over each user-item pair in the training set
    for user_id, user_ratings in train_matrix.iterrows():
        for movie_id, rating in user_ratings.items():
            
            # Skip if the rating is non-zero (indicating a rating given by the user)
            if rating != 0:
                continue
            
            # Check if the movie has a neighborhood defined
            if movie_id in item_neighborhoods:
                neighborhood = item_neighborhoods[movie_id]
                
                # Initialize a flag to track if the rating has been predicted
                rating_predicted = False
                
                # Iterate over the items in the neighborhood
                for neighbor_movie_id in neighborhood:
                    # Check if the neighbor movie has been rated by the user
                    if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0:
                        # Use the rating of the closest item in the neighborhood
                        predicted_rating = user_ratings.loc[neighbor_movie_id]
                        rating_predicted = True
                        break
                
                # If none of the items in the neighborhood have been rated, use the user's average rating
                if not rating_predicted:
                    if user_ratings[user_ratings != 0].empty:
                        # If the user hasn't rated any movies, use the global mean rating
                        predicted_rating = train_matrix[train_matrix != 0].mean().mean()
                    else:
                        # Use the average of the ratings given by the user
                        predicted_rating = user_ratings[user_ratings != 0].mean()
                
                # Assign the predicted rating to the corresponding cell in the DataFrame
                predicted_ratings.at[user_id, movie_id] = predicted_rating

    # Fill NaN values with mean ratings across all users
    predicted_ratings.fillna(predicted_ratings.mean().mean(), inplace=True)

    # Set the index of the DataFrame as the user ID of the train set
    predicted_ratings.index = train_matrix.index

    # Display the head of the predicted_ratings DataFrame
    print(predicted_ratings.head())

    return predicted_ratings

# Example usage:
# Assuming you have the train matrix, item neighborhood dictionary, and movie_id_to_index dictionary
# train_matrix, item_neighborhoods, movie_id_to_index = ...

# Recommend movies for each user in the train set
predicted_ratings_classification_train = recommend_movies(train_matrix, item_neighborhoods_classification_train, movie_id_to_index)

# Print the shape of the predicted ratings matrix
print("Shape of predicted ratings matrix:", predicted_ratings_classification_train.shape)


movieId    1         4         9         11        13        15        18      \
userId                                                                          
53       5.000000  5.000000  5.000000  5.000000  5.000000  5.000000  5.000000   
578      4.187500  4.187500  4.187500  4.187500  4.187500  4.187500  4.187500   
165      3.400000  3.400000  3.400000  3.400000  3.400000  3.400000  3.400000   
362      4.137931  4.137931  4.137931  4.137931  4.137931  4.137931  4.137931   
308      2.555556  2.555556  2.555556  2.555556  2.555556  2.555556  2.555556   

movieId    20        24        30      ...    185473    186587    187031  \
userId                                 ...                                 
53       5.000000  5.000000  5.000000  ...  5.000000  5.000000  5.000000   
578      4.187500  4.187500  4.187500  ...  4.187500  4.187500  4.187500   
165      3.400000  3.400000  3.400000  ...  3.400000  3.400000  3.400000   
362      4.137931  4.137931  4.137931  ...  4.137931

C:\Users\Jaume\AppData\Local\Temp\ipykernel_8932\1893350747.py:61: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings.fillna(predicted_ratings.mean().mean(), inplace=True)


In [ ]:
type(train_matrix)

pandas.core.frame.DataFrame

In [ ]:
predicted_ratings_classification_train

movieId,1,4,9,11,13,15,18,20,24,30,...,185473,186587,187031,187541,188675,188797,191005,193573,193583,193587
userId,,,,,,,,,,,,,,,,,,,,,
53,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
578,4.187500,4.187500,4.187500,4.187500,4.187500,4.187500,4.187500,4.187500,4.187500,4.187500,...,4.187500,4.187500,4.187500,4.187500,4.187500,4.187500,4.187500,4.187500,4.187500,4.187500
165,3.400000,3.400000,3.400000,3.400000,3.400000,3.400000,3.400000,3.400000,3.400000,3.400000,...,3.400000,3.400000,3.400000,3.400000,3.400000,3.400000,3.400000,3.400000,3.400000,3.400000
362,4.137931,4.137931,4.137931,4.137931,4.137931,4.137931,4.137931,4.137931,4.137931,4.137931,...,4.137931,4.137931,4.137931,4.137931,4.137931,4.137931,4.137931,4.137931,4.137931,4.137931
308,2.555556,2.555556,2.555556,2.555556,2.555556,2.555556,2.555556,2.555556,2.555556,2.555556,...,2.555556,2.555556,2.555556,2.555556,2.555556,2.555556,2.555556,2.555556,2.555556,2.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,3.606346,3.261146,3.261146,3.606346,3.261146,3.261146,3.261146,3.261146,3.261146,3.261146,...,3.261146,3.261146,3.261146,3.261146,3.261146,3.261146,3.261146,3.261146,3.261146,3.261146
225,4.047619,4.047619,4.047619,4.047619,4.047619,4.047619,4.047619,4.047619,4.047619,4.047619,...,4.047619,4.047619,4.047619,4.047619,4.047619,4.047619,4.047619,4.047619,4.047619,4.047619
52,4.467742,4.467742,4.467742,4.467742,4.467742,4.467742,4.467742,4.467742,4.467742,4.467742,...,4.467742,4.467742,4.467742,4.467742,4.467742,4.467742,4.467742,4.467742,4.467742,4.467742


### Step 4: Recommendations Generation

In [ ]:
import pandas as pd

# Convert predicted_ratings_classification_train to a pandas DataFrame if it's a numpy array
if isinstance(predicted_ratings_classification_train, np.ndarray):
    # Use the length of train_matrix as the number of rows and items as columns
    num_users, num_items = train_matrix.shape
    predicted_ratings_classification_train = pd.DataFrame(predicted_ratings_classification_train, index=range(num_users), columns=range(num_items))

# Initialize a dictionary to store top recommendations for each user
top_recommendations_per_user = {}

# Iterate over each user in the train matrix
for user_id in train_matrix.index:
    # Get the top recommendations for the current user using predicted_ratings_classification_train
    recommendations = get_top_recommendations(user_id, predicted_ratings_classification_train, df)
    
    # Store the recommendations in the dictionary
    top_recommendations_per_user[user_id] = recommendations

# Print the top 5 recommendations for each user
for user_id, recommendations in top_recommendations_per_user.items():
    print(f"User {user_id} recommendations:", recommendations)


User 53 recommendations: ['Toy Story ', 'Vie en Rose, La (Môme, La) ', 'How the Grinch Stole Christmas! ', 'Disturbia ', 'Sunshine ']
User 578 recommendations: ['Toy Story ', 'Grindhouse ', 'Hoax, The ', 'Black Book (Zwartboek) ', 'Sunshine ']
User 165 recommendations: ['Toy Story ', 'Vacancy ', 'In the Land of Women ', 'Vie en Rose, La (Môme, La) ', "It's a Very Merry Muppet Christmas Movie "]
User 362 recommendations: ['Toy Story ', 'Vacancy ', 'In the Land of Women ', 'Vie en Rose, La (Môme, La) ', "It's a Very Merry Muppet Christmas Movie "]
User 308 recommendations: ['Toy Story ', 'Vie en Rose, La (Môme, La) ', 'Fracture ', 'Hoax, The ', 'Disturbia ']
User 475 recommendations: ['Room for Romeo Brass, A ', 'Creep ', 'Toy Story ', 'Rent ', 'Wolf Creek ']
User 460 recommendations: ['Vie en Rose, La (Môme, La) ', 'Fracture ', 'Hoax, The ', 'Waiting to Exhale ', 'Disturbia ']
User 90 recommendations: ['21 Up ', 'In the Land of Women ', 'Maniac Cop ', 'Sharkwater ', 'Waiting to Exhale '

### Step 5: Model Evaluation (Item KNN Classification)

In [ ]:
import numpy as np
import pandas as pd

# Assuming predicted_ratings_classification_train is your predicted ratings DataFrame or numpy array

# Ensure predicted_ratings_classification_train is a numpy array
if isinstance(predicted_ratings_classification_train, pd.DataFrame):
    predicted_ratings_classification_train = predicted_ratings_classification_train.to_numpy()

# Assuming train_matrix is the numpy array representing the item-user matrix

# Ensure train_matrix is a 2D array
if isinstance(train_matrix, pd.DataFrame):
    train_matrix = train_matrix.to_numpy()

if train_matrix.ndim == 1:
    train_matrix = np.expand_dims(train_matrix, axis=0)

# Evaluate the model
mae, rmse = evaluate_model(train, predicted_ratings_classification_train)

print("Mean Absolute Error (MAE) on Training Data:", mae)
print("Root Mean Squared Error (RMSE) on Training Data:", rmse)


Mean Absolute Error (MAE) on Training Data: 0.8206408720741247
Root Mean Squared Error (RMSE) on Training Data: 1.0386863306664693


### Step 6: Parameter Tuning

Experiment with different parameters such as similarity threshold, neighborhood size, and similarity metric to optimize the performance of your ItemKNN algorithm.
Use techniques like cross-validation to tune these parameters and avoid overfitting.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import make_scorer
import numpy as np

# Define a custom scorer based on cosine similarity defined above
cosine_similarity_scorer = make_scorer(cosine_similarity)

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 30, 40],
    'metric': ['cosine', 'euclidean']
}

# Initialize NearestNeighbors model
knn_model = NearestNeighbors()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring=cosine_similarity_scorer)

# Fit the data to perform hyperparameter tuning
grid_search.fit(train_matrix)  # val_matrix contains item-user matrix

# Get the best hyperparameters
best_params = grid_search.best_params_

print("Best parameters:", best_params)


c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

GridSearchCV(cv=5, estimator=NearestNeighbors(),
             param_grid={'metric': ['cosine', 'euclidean'],
                         'n_neighbors': [5, 10, 15, 30, 40]},
             scoring=make_scorer(cosine_similarity, response_method='predict'))

Best parameters: {'metric': 'cosine', 'n_neighbors': 5}


We are now going to obtain again the ratings for the full val set to evaluate the performance. 

In [ ]:
# Assuming you have the best model after hyperparameter tuning
# best_model = ...

# Function to recalculate predictions for a given matrix using the trained model
def recalculate_predictions(matrix, item_neighborhoods):
    """
    Recalculate predictions for a given matrix using the trained model.

    Parameters:
        matrix (pd.DataFrame): Matrix containing user-item ratings.

    Returns:
        pd.DataFrame: DataFrame containing recalculated predicted ratings for each user.
    """
    # Initialize an empty DataFrame to store recalculated predicted ratings
    recalculated_predictions = pd.DataFrame(index=matrix.index, columns=matrix.columns)

    # Iterate over each user-item pair in the matrix
    for user_id, user_ratings in matrix.iterrows():
        for movie_id, rating in user_ratings.items():
            # Skip if the rating is non-zero (indicating a rating given by the user)
            if rating != 0:
                continue

            # Check if the movie has a neighborhood defined
            if movie_id in item_neighborhoods:
                neighborhood = item_neighborhoods[movie_id]

                # Initialize a flag to track if the rating has been predicted
                rating_predicted = False

                # Iterate over the items in the neighborhood
                for neighbor_movie_id in neighborhood:
                    # Check if the neighbor movie has been rated by the user
                    if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0:
                        # Use the rating of the closest item in the neighborhood
                        predicted_rating = user_ratings.loc[neighbor_movie_id]
                        rating_predicted = True
                        break

                # If none of the items in the neighborhood have been rated, use the user's average rating
                if not rating_predicted:
                    if user_ratings[user_ratings != 0].empty:
                        # If the user hasn't rated any movies, use the global mean rating
                        predicted_rating = matrix[matrix != 0].mean().mean()
                    else:
                        # Use the average of the ratings given by the user
                        predicted_rating = user_ratings[user_ratings != 0].mean()

                # Assign the predicted rating to the corresponding cell in the DataFrame
                recalculated_predictions.at[user_id, movie_id] = predicted_rating

    # Fill NaN values with mean ratings across all users
    recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)

    return recalculated_predictions


# Recalculate predictions for the validation set (train_val_matrix)
recalculated_val_predictions = recalculate_predictions(val_matrix, item_neighborhoods_classification_train)


C:\Users\Jaume\AppData\Local\Temp\ipykernel_8932\4246365781.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)


In [ ]:
recalculated_val_predictions

movieId,1,4,9,11,13,15,18,20,24,30,...,185473,186587,187031,187541,188675,188797,191005,193573,193583,193587
userId,,,,,,,,,,,,,,,,,,,,,
270,3.632564,3.625000,3.632564,3.625000,3.625000,3.625000,3.625000,3.625000,3.625000,3.625000,...,3.625000,3.625000,3.625000,3.625000,3.625000,3.625000,3.625000,3.625000,3.625000,3.625000
124,3.632564,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250,...,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250,4.156250
368,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,...,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224,2.776224
213,3.632564,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,...,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667,3.616667
63,3.632564,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,...,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423,3.570423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,3.632564,3.464286,3.464286,3.464286,3.464286,3.464286,3.464286,3.464286,3.464286,3.464286,...,3.464286,3.464286,3.464286,3.464286,3.464286,3.464286,3.464286,3.464286,3.464286,3.464286
268,3.263158,3.263158,3.263158,3.263158,3.263158,3.263158,3.263158,3.263158,3.263158,3.263158,...,3.263158,3.263158,3.263158,3.263158,3.263158,3.263158,3.263158,3.263158,3.263158,3.263158
455,3.562500,3.562500,3.562500,3.632564,3.562500,3.562500,3.562500,3.562500,3.562500,3.562500,...,3.562500,3.562500,3.562500,3.562500,3.562500,3.562500,3.562500,3.562500,3.562500,3.562500


In [ ]:
# Evaluate the model
mae, rmse = evaluate_model(val, recalculated_val_predictions)

print("Mean Absolute Error (MAE) on Train-Validation Data:", mae)
print("Root Mean Squared Error (RMSE) on Train-Validation Data:", rmse)

Mean Absolute Error (MAE) on Train-Validation Data: 0.8301102661152354
Root Mean Squared Error (RMSE) on Train-Validation Data: 1.0480506291702596


### Step 7: Deployment

Once we have all the metrics, we have to recalculate the ratings for the test set, we are going to repeat the same process for the test set.

In [ ]:
# We have to reobtain the neighborhood for the train_val data
# We will use the already calculated item-item similarity matrix for train_val data
item_neighborhoods_classification_train_val = item_neighborhood_selection(item_similarity_matrix_train_val, k=5, item_ids=index_to_movie_id)

# Recalculate predictions for the validation set (test matrix)
recalculated_test_predictions = recalculate_predictions(test_matrix, item_neighborhoods_classification_train_val)

C:\Users\Jaume\AppData\Local\Temp\ipykernel_8932\4246365781.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)


We also have to evaluate the metrics for the test set.

In [ ]:
# Evaluate the model
mae, rmse = evaluate_model(test, recalculated_test_predictions)

print("Mean Absolute Error (MAE) on Test Data:", mae)
print("Root Mean Squared Error (RMSE) on Test Data:", rmse)

Mean Absolute Error (MAE) on Test Data: 0.8803816981703679
Root Mean Squared Error (RMSE) on Test Data: 1.118645339273174
